# Imports

In [1]:
import numpy
import itertools
from sympy import primerange

## Prime Number Generation

In [2]:
# Sieve of Eratosthenes
def sieve_of_eratosthenes(k):
    prime_numbers = []  # List to store prime numbers
    n = 2  # Start with the first prime number
    
    while len(prime_numbers) < k:
        is_prime = True  # Assume n is prime initially
        
        # Check if n is divisible by any previous prime number
        for prime in prime_numbers:
            if n % prime == 0:
                is_prime = False
                break
        
        if is_prime:
            prime_numbers.append(n)  # Add n to the list of prime numbers
        
        n += 1  # Move to the next number
        
    return prime_numbers



In [3]:
# Croft's Spiral Sieve
def croft():
    # Implementation is based on erat3 from here:
    #   http://stackoverflow.com/q/2211990
    # and this website:
    #   http://www.primesdemystified.com/
    # Memory usage increases roughly linearly with the number of primes seen.
    # dict ``roots`` stores an entry p**2:p for every prime p.
    for p in (2, 3, 5):
        yield p
    roots = {9: 3, 25: 5}  # Map d**2 -> d.
    primeroots = frozenset((1, 7, 11, 13, 17, 19, 23, 29))
    selectors = (1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0)
    for q in itertools.compress(
            # Iterate over prime candidates 7, 9, 11, 13, ...
            itertools.islice(itertools.count(7), 0, None, 2),
            # Mask out those that can't possibly be prime.
            itertools.cycle(selectors)
    ):
        # Using dict membership testing instead of pop gives a
        # 5-10% speedup over the first three million primes.
        if q in roots:
            p = roots[q]
            del roots[q]
            x = q + 2*p
            while x in roots or (x % 30) not in primeroots:
                x += 2*p
            roots[x] = p
        else:

            roots[q*q] = q
            yield q
    pass

def primes_list(limit):
    croft_gen = croft()
    return [next(croft_gen) for _ in range(limit)]


In [4]:
# Using primerange from sympy
def sympy_prime(k):
    return list(primerange(1, k))

### Timing the prime number generation

In [5]:
%%timeit
k = 1000  # Generate the first 10000 prime numbers 
primes_sieve = sieve_of_eratosthenes(k)

12.7 ms ± 31.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
%%timeit
k = 1000  # Generate the first 10000 prime numbers 
primes_croft = primes_list(k)

481 µs ± 4.73 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [7]:
%%timeit
k = 1000 
primes_sympy = sympy_prime(k)

202 µs ± 770 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Time Comparision for prime number generation

k denotes the number of primes

| Sr. No | k | Eratosthenes | Croft | Sympy |
|:--:|:--------:|:--------:|:--------:|:--------:|
| 1 |  100 | 132us       | 19.5us   | 17.7us     |
| 2 | 1,000  | 10.8ms    | 416us   | 140us     |
| 3 | 10,000  | 1.02s    | 6.59ms   | 1.8ms     |
| 4 | 50,000  | 25.8s    | 59.9ms   | 24.3ms     |
| 5 | 100,000  | 98s    | 112ms   | 51ms     |
| 6 | 125,000  | 169s    | 151ms   | 67.7ms     |
| 7 | 150,000  | 242s    | 180us   | 82.7ms     |

## Implementing Computation Algorithms

In [8]:
# Algorithm-1
def algo1(pl): # pl is list of primes
    sum_fact = 0  # Sum of factorials
    fact_num = 1  # Factorial
    for i in range(2, pl[-1]+1):
        fact_num *= (i-1)
        sum_fact += fact_num**2
        if i in pl and sum_fact % i == 0:
            print(True, i) # Print the result as soon as we get the required p
            
    #print("Search Concluded.")

In [9]:
# Algorithm-2
def algo2(pl): # pl is list of primes
    sum_fact = 0  # Sum of factorials
    fact_num = 1  # Factorial
    for i in range(2, pl[-1]+1):
        fact_num *= (i-1)**2
        sum_fact += fact_num
        if i in pl and sum_fact % i == 0:
            print(True, i) # Print the result as soon as we get the required p
            
    #print("Search Concluded.")

In [11]:
n=1000
_ = primes_list(n)

## Timing the two algorithms

In [12]:
%%timeit
algo1(_)

4.72 s ± 28.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
%%timeit
algo2(_)

117 ms ± 312 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Time Comparision for algorithm run-times

k denotes the number of primes

| Sr. No | k | Algorithm-1 | Algorithm-2 |
|:--:|:--------:|:--------:|:--------:|
| 1 | 1000 | 4.72s  | 117ms|
| 2 | 5,000  |  290s    | 2.51s   |
| 3 | 10,000  | 2412s    | 13.5s   |
| 4 | 15,000  | 8478s    | 34s   | 
| 5 | 16,384  | 10849s    | 45.2s   |
| 6 | 25,000 | --- | 117s |
| 7 | 50,000 | --- | 561s |
| 8 | 75,000 | --- | 1651s |
| 9 | 100,000 | --- | 51min 57s |
| 10 | 125,000 | --- | 1h 22min 14s |
| 11 | 150,000 | --- | 2h 8min 30s |
